In [ ]:
%pip install -q \
pypdf \
faiss-cpu \
"langchain>=0.3,<0.4" \
"langchain-core>=0.3,<0.4" \
"langchain-text-splitters>=0.3,<0.4" \
langchain-community \
langchain_nvidia_ai_endpoints \
openai \
numpy==1.26.4
print("Dependencies installed successfully.")


In [1]:
API_KEY = "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" # Paste your actual API key here

In [7]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings, NVIDIARerank
from langchain_nvidia_ai_endpoints import NVIDIARerank
from openai import OpenAI
# Initialize the Embeddings client
embedding_client = NVIDIAEmbeddings(
model="nvidia/nv-embed-v1",
api_key=API_KEY,
truncate="NONE",
)
# Initialize Chat client for LLM generation
gpt_client = ChatNVIDIA(
model="meta/llama-3.2-1b-instruct",
api_key=API_KEY,
temperature=0.2,
top_p=0.5,
)
# Initialize the reranker client
reranker_client = NVIDIAReRank(
model="nvidia/llama-3.2-nv-rerankqa-1b-v2",
api_key=API_KEY,
)
# Initialize Topic Control client
topic_control_client = OpenAI(
base_url="https://integrate.api.nvidia.com/v1",
api_key=API_KEY,
)
print("NVIDIA clients initialized successfully.")

NameError: name 'NVIDIAReRank' is not defined

In [9]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings, NVIDIARerank
from openai import OpenAI

# Initialize the Embeddings client
embedding_client = NVIDIAEmbeddings(
    model="nvidia/nv-embed-v1",
    api_key=API_KEY,
    truncate="NONE",
)

# Initialize Chat client for LLM generation
gpt_client = ChatNVIDIA(
    model="meta/llama-3.2-1b-instruct",
    api_key=API_KEY,
    temperature=0.2,
    top_p=0.5,
)

# FIXED: Changed NVIDIAReRank to NVIDIARerank to match the import
reranker_client = NVIDIARerank(
    model="nvidia/llama-3.2-nv-rerankqa-1b-v2",
    api_key=API_KEY,
)

# Initialize Topic Control client
topic_control_client = OpenAI(
    base_url="https://integrate.api.nvidia.com/v1",
    api_key=API_KEY,
)

print("NVIDIA clients initialized successfully.")

NVIDIA clients initialized successfully.


In [ ]:
pip install -U langchain-nvidia-ai-endpoints

In [11]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings, NVIDIARerank

from openai import OpenAI
# Initialize the Embeddings client
embedding_client = NVIDIAEmbeddings(
model="nvidia/nv-embed-v1",
api_key=API_KEY,
truncate="NONE",
)
# Initialize Chat client for LLM generation
gpt_client = ChatNVIDIA(
model="meta/llama-3.2-1b-instruct",
api_key=API_KEY,
temperature=0.2,
top_p=0.5,
)
# Initialize the reranker client
reranker_client = NVIDIARerank(
model="nvidia/llama-3.2-nv-rerankqa-1b-v2",
api_key=API_KEY,
)
# Initialize Topic Control client
topic_control_client = OpenAI(
base_url="https://integrate.api.nvidia.com/v1",
api_key=API_KEY,
)
print("NVIDIA clients initialized successfully.")

NVIDIA clients initialized successfully.


In [13]:
import os
print(f"Create your 'pdf' folder here: {os.getcwd()}")

Create your 'pdf' folder here: C:\Users\nairvis\Downloads\PCAAI Training


In [15]:
from langchain.document_loaders import PyPDFDirectoryLoader
loader = PyPDFDirectoryLoader("./pdf")
data = loader.load()

In [21]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
chunk_size=800,
chunk_overlap=60,
)
documents = text_splitter.split_documents(data)

In [19]:
import numpy as npa
from langchain.vectorstores import FAISS
docsearch = FAISS.from_documents(documents, embedding=embedding_client)
docsearch.save_local(folder_path="/embeddings")

NameError: name 'documents' is not defined

In [ ]:
from langchain.vectorstores import FAISS
docsearch = FAISS.load_local(
"/embeddings", embedding_client, allow_dangerous_deserialization=True
)

In [23]:
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = PyPDFDirectoryLoader("./pdf")
data = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=60,
)
documents = text_splitter.split_documents(data) # Variable defined here
print(f"Total document chunks created: {len(documents)}")

Total document chunks created: 66


In [27]:
from langchain_community.vectorstores import FAISS
import numpy as np
docsearch = FAISS.from_documents(documents, embedding=embedding_client)
docsearch.save_local("embeddings_db") # Using a relative folder name
print("FAISS index saved successfully.")


FAISS index saved successfully.


In [29]:
retriever = docsearch.as_retriever(search_kwargs={"k": 20})


In [31]:
HR_TOPIC_CONTROL_PROMPT = (
"""You are an HR assistant for InnovateSphere. Only answer questions about HR policies,
employee handbook, leave entitlements, Code of Conduct, diversity, workplace guidelines, or
performance review procedures. Do not answer any queries about marketing, sales, products,
financials, or campaigns. If the query is outside HR topics, respond that you cannot provide such
information."""
)
MARKETING_TOPIC_CONTROL_PROMPT = (
"""You are a marketing assistant for InnovateSphere. Only answer questions about
InnovateSphere's marketing campaigns, brand guidelines, messaging, target markets, and social
media policies. Do not address HR policies, sales results, product information, or internal staff
matters. Respond with a polite rejection if the question is outside marketing topics."""
)
SALES_TOPIC_CONTROL_PROMPT = (
"""You are a sales assistant for InnovateSphere. Provide information only about sales reports,
product catalog, sales team directory, and quarterly performance. Never answer questions
relating to HR, marketing, or internal policies. If the query is not about sales, reject it as outside
sales scope."""
)

In [33]:
import os
print(os.path.exists('RAGPipeline.py')) # Should return True

True


In [43]:
import os
print(f"Create your 'pdf' folder here: {os.getcwd()}")

import RAGPipeline
topic_control_model = "nvidia/llama-3.1-nemoguard-8b-topic-control"
pipeline = RAGPipeline.RAGPipeline(
retriever=retriever,
gpt_client=gpt_client,
reranker_client=reranker_client,
topic_control_client=topic_control_client,
topic_control_model=topic_control_model,
)

Create your 'pdf' folder here: C:\Users\nairvis\Downloads\PCAAI Training
RAGPipeline initialized with model: nvidia/llama-3.1-nemoguard-8b-topic-control


In [45]:
pipeline.query("What is the professional integrity statement of InnovateSphere?")

AttributeError: 'RAGPipeline' object has no attribute 'query'

In [72]:
import importlib
import RAGPipeline
importlib.reload(RAGPipeline) # This forces the update without a full kernel restart

# Now re-initialize your pipeline
pipeline = RAGPipeline.RAGPipeline(
    retriever=retriever,
    gpt_client=gpt_client,
    reranker_client=reranker_client,
    topic_control_client=topic_control_client,
    topic_control_model=topic_control_model
)

In [64]:
pipeline.query("What is the professional integrity statement of InnovateSphere?")

'The professional integrity statement of InnovateSphere is:\n\n"We are committed to acting with integrity in all our business dealings, including mergers and acquisitions, and to providing our clients with the highest level of service and expertise, while maintaining the highest standards of professionalism and ethics."'

In [56]:
pipeline.query("Show the Q1 sales performance figures.")

'Here are the Q1 sales performance figures mentioned in the transcript:\n\n* eBay paid $3.9 billion for Skype\n* eBay paid $2.5 billion upfront and a $1.4 billion earn-out for Skype\n* Oracle, Intel, Google, Amazon, and GE are all mentioned as having market caps above $100 billion\n* The study is keyed around a new metric called "Buyer Power Ratio" (BPR), which is the buyer\'s market cap divided by the size of the deal.'

In [58]:
pipeline.query("Describe InnovateSphere's brand identity guidelines.")

'Based on the provided transcript, InnovateSphere\'s brand identity guidelines appear to be centered around a collaborative and strategic approach to M&A negotiations. Here are some key elements that can be inferred from the transcript:\n\n1. **Focus on maximizing revenue and EBIT**: The transcript mentions that InnovateSphere\'s clients are looking to maximize revenue and EBIT, suggesting a focus on value creation and strategic partnerships.\n2. **Use of advanced draft of study**: The transcript highlights the use of an advanced draft of a study, which suggests a commitment to innovation and best practices in M&A negotiations.\n3. **Collaborative approach**: The transcript mentions that InnovateSphere is a joint effort between the ABA and SRS Aqueon, indicating a collaborative approach to M&A negotiations.\n4. **Focus on due diligence and representation**: The transcript mentions that InnovateSphere will provide due diligence and representation to clients, suggesting a focus on suppor

In [66]:
pipeline.topic_control_prompt = HR_TOPIC_CONTROL_PROMPT
pipeline.query("How many days of annual leave do employees receive?",
enable_topic_control=True)

Running topic control check...


'There is no mention of employee leave in the conversation. The conversation is between Keith L. and Rick K., discussing a potential deal between their clients and a company.'

In [76]:
pipeline.topic_control_prompt = HR_TOPIC_CONTROL_PROMPT
pipeline.query("Provide the product catalog details.", enable_topic_control=True)

Running topic control check...


'Based on the conversation, it appears that the product catalog details are not explicitly mentioned. However, the conversation mentions that the product catalog is a bundle of products, and that the client is interested in acquiring a product that is not yet to market.\n\nThe conversation also mentions that the client has a list price for the product, and that the product is being sold in a bundle. Additionally, the conversation mentions that the client has a "product you sell it one way there\'s a list price and everybody knows what" which suggests that the product is being sold in a bundle.\n\nTherefore, it is likely that the product catalog details include:\n\n* A list price for the product\n* A bundle of products\n* A product catalog that lists the products in the bundle\n* Information about the product\'s features, pricing, and availability\n\nSome specific details that may be included in the product catalog are:\n\n* Product names and descriptions\n* Product features and benefit

In [78]:
pipeline.query("Detail performance review criteria for staff.",
enable_rerank=True)

Reranking documents...


'Based on the conversation, it seems that the speaker is considering a revenue-based turnover metric, which is a simple and measurable metric that aligns incentives with the company\'s goals. However, the speaker is also aware of the potential drawbacks of using this metric, such as:\n\n* It may not align incentives with the company\'s goals, as the former shareholders may be more focused on short-term profits than long-term growth.\n* It may lead to a "flimsy revenues" or "unprofitable revenues" culture, where the company prioritizes short-term gains over long-term sustainability.\n\nTo address these concerns, the speaker proposes to:\n\n* Negotiate a revenue-based turnover metric that is more aligned with the company\'s goals.\n* Define the key employees who will be responsible for running the target and ensure that they remain in control for a reasonable period of time.\n* Include language in the agreement that outlines the expectations for the business, including the goal of maximi

In [80]:
pipeline.topic_control_prompt = HR_TOPIC_CONTROL_PROMPT
pipeline.query("Explain the diversity commitment at InnovateSphere.",
enable_topic_control=True, enable_rerank=True)

Running topic control check...
Reranking documents...


"The diversity commitment at InnovateSphere is not explicitly stated in the provided text. However, based on the context, it appears that InnovateSphere is a company that values diversity and inclusion, as evidenced by the discussion about the earn-out formulation and the consideration of a $30 per share price.\n\nThe text mentions that the company's clients have decided on a price of $30 per share, which is a relatively high price, suggesting that InnovateSphere is committed to valuing its clients' shares. Additionally, the discussion about the earn-out formulation and the consideration of a $30 per share price implies that the company is willing to invest in its clients' growth and success.\n\nFurthermore, the text mentions that the company is willing to invest in its clients' employees, licenses, and contracts, which suggests that InnovateSphere is committed to supporting its clients' growth and success.\n\nOverall, the diversity commitment at InnovateSphere appears to be a core val

In [82]:
pipeline.topic_control_prompt = MARKETING_TOPIC_CONTROL_PROMPT
pipeline.query("What are the brand usage rules?", enable_topic_control=True,
enable_rerank=True)

Running topic control check...
Reranking documents...


'The brand usage rules mentioned in the speaker\'s presentation are:\n\n1. "Drop a little tiny bit of your technology in there" - This implies that the speaker wants to use a small amount of their technology to negotiate a deal.\n2. "Should you get full or an hour credit for that" - This suggests that the speaker wants to negotiate a deal with a full credit, rather than just a partial credit.\n3. "We should but we\'re going to negotiate that so it\'s not going to be simple if you go to the next slide" - This implies that the speaker wants to negotiate the deal in a way that is more complex and nuanced than simply accepting a full credit.\n4. "How should revenues be allocated if the products sold in a bundle" - This suggests that the speaker wants to negotiate the deal in a way that takes into account the revenue generated by selling products in bundles.\n5. "If your client uses my clients that has two product as a loss leader it gives it away so it could sell some of your existing prod

In [84]:
pipeline.topic_control_prompt = SALES_TOPIC_CONTROL_PROMPT
pipeline.query("Provide the product catalog details", enable_topic_control=True,
enable_rerank=True)

Running topic control check...
Reranking documents...


'There is no product catalog mentioned in the transcript. The conversation appears to be about negotiating a deal with a client, specifically discussing revenue allocation, measurement periods, and key employees.'